In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install scipy

In [ ]:
pip install numpy

In [ ]:
pip install seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
from numpy.random import randn
import seaborn as sns
from scipy import stats

%matplotlib inline

In [ ]:
import os
import os.path

Загрузим данные о шахматных встречах из csv файла

In [ ]:
train = pd.read_csv('./games.csv')
test = pd.read_csv('./test.csv')

просмотрим количество строк и колонок для количественной оценки данных

In [ ]:
train.shape

In [ ]:
просмотрим наименования колонок с которыми будем работать

In [ ]:
train.columns

Просмотрим первые строки для оценки типов данных и их наличия

In [ ]:
train.head()

В данной модели нас интересует влияние очков рейтинга на исход встречи

Для этого создадим поля для численной оценки:
1) rate_difference которое описывает модуль разности рейтинга игроков(колонка созданна для того чтобы была возможность оценить встречи по разнице играющих, что безусловно можно делать и без данной колонки, вычисляя на каждом этапе данное число)
2) winner_rate_difference которое описывает разность рейтинга победителя(колонка созданна для того чтобы была возможность оценить исход встречи при заданной разности рейтинга игроков, что безусловно можно делать и без данной колонки, вычисляя на каждом этапе данное число)
3) численная оценка честности матча(так как необходимых оценок встреч среди данных нет, було решено основываясь на личном опыте игры в шахматы создать колонку численной оценки встречи, при разнице рейтинга более чем в 100 очков зачастую победить не представляется возможным)

и посмотрим на получившиеся данные

In [ ]:
train['rate_difference'] = np.abs(train['white_rating'] - train['black_rating'])
train['winner_rate_difference'] = train.apply(lambda x: ((x['white_rating'] - x['black_rating'] if x['winner'] == 'white' else x['black_rating'] - x['white_rating']) if x['winner'] != 'draw' else 0), axis=1)
train['game_qual'] = train.apply(lambda x: (int(x['winner_rate_difference'] / 10) if (abs(x['winner_rate_difference']) / 10) <= 9 else 9 ), axis=1)
X_train = train.drop(labels=['winner'], axis=1).copy()
Y_train = train['winner'].values.copy()
train.head()

Проверим изменения в количестве данных

In [ ]:
X_train.shape, Y_train.shape

Выведем гистограммы для первичного анализа данных

In [ ]:
train.hist(figsize=(16, 20), bins=50, xlabelsize=8,ylabelsize=8);

Поскольку мы оцениваем влияние разности рейтинга на исход в стречи то целевой столбец rate_difference

расмотрим детальнее гистограмма по данному столбцу

заметим что ярких явных выбрасов не наблюдается

In [ ]:
train['rate_difference'].hist(figsize=(16, 20), bins=50, xlabelsize=8,ylabelsize=8);

для дальнейшего анализа построим график rate_difference/winner_rate_difference

и заметим линейность(действительно ведь одни данные состоят из других...)

но нас интересует больше знак этой зависимости, который указывает нам на исход встречи
а также сама разность рейтингов

на данном этапе уже заметны выбросы достаточно странных данных которые указывают что с разностью рейтингов более 100 побеждает менее опытный игрок

а также то что игроки с разницей рейтингов более 100 выходят из встречи в ничью

такие данные исходя из описанного выше считаем выбросами

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5))
ax.scatter(x=train['rate_difference'], y=train['winner_rate_difference'])
plt.ylabel('winner_rate_difference', fontsize=13)
plt.xlabel('rate_difference', fontsize=13)
plt.show()

далее расмотрм тот же график но отсечем те данные которые посчитали не валидными на предыдущем этапе

теперь график зависимости результата встречи от разницы рейтинга играющих является более правдоподобным
поскольку как правило людам с рейтингом выше нашего на 100 значительно легче выйграть

In [ ]:
train = train.drop(
    train[
        (
            (train['black_rating'] + 100 < train['white_rating']) & (train['winner'] == 'black')
        ) | 
        (
            (train['black_rating']  > train['white_rating'] + 100) & (train['winner'] == 'white')
        ) |
        (
            (train['rate_difference'] > 100) & (train['winner'] == 'draw')
        )
    ].index
)

fig, ax = plt.subplots(figsize=(9, 5))
ax.scatter(x=train['rate_difference'], y=train['winner_rate_difference'])
plt.ylabel('winner_rate_difference', fontsize=13)
plt.xlabel('rate_difference', fontsize=13)
plt.show()

Далее посмотрим на график оценки к разнице рейтинга(И да не стоит забывать что эти поля имеют 1 общего родителя)

мы видим относительно предметной области хороший график без явных выбросов(что кажется логичным...)

Но обратим внимание на самый яркий столбец, он имеет такой вид ввиду того что в данной предметной области чем больше разность рейтингов тем более легкая победа будет, такой картины в отрицательных значених мы не наблюдаем ввиду того что приняли постфактом, то что победить человеку с рейтингом ниже более чем на 100 очков невозможно(отфильтровали данные выше...)

данный график нас устраивает

In [ ]:
columns = ['rate_difference', 'winner_rate_difference', 'game_qual']

g = sns.FacetGrid(train[columns], hue='game_qual', height=5) \
    .map(plt.scatter, 'game_qual', 'rate_difference') \
    .add_legend()

g = g.map(plt.scatter, 'game_qual', 'rate_difference', edgecolor='w').add_legend()

plt.show()

Графически оценим нормализацию наших данных

исходя из графиков видно, что данные имеют на небольшом участке нормализованный вид, но по большей части нет

In [ ]:
sns.distplot(train['rate_difference'], fit=stats.norm);

(mu, sigma) = stats.norm.fit(train['rate_difference'])

plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Rate difference distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['rate_difference'], plot=plt)
plt.show()

Прологарифмируем данные в попытке нормализовать их

In [ ]:
train['rate_difference'] = np.log1p(train['rate_difference'])

проверим умеличилась ли протяженность наложения наших графиков

Да увеличилось, что позволяет нам сказать что на данном промежутке наши данные нормализованы
но также мы видим некоторые выбросы... связаны они с набором данных в данном случае из-за наличия игр соперников координально отличающихся по рейтингу, что в обычных играх не встречается

In [ ]:
sns.distplot(train['rate_difference'], fit=stats.norm);

(mu, sigma) = stats.norm.fit(train['rate_difference'])

plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Rate difference distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['rate_difference'], plot=plt)
plt.show()

Посмотрим на отображение полей друг на друга

Видим их всех взаимосвязь(как и предполагалось)

In [ ]:
columns = ['rate_difference', 'winner_rate_difference', 'game_qual']

pd.plotting.scatter_matrix(train[columns], figsize=(12,12))
plt.figure()

Для более точного понимания степени связанности данных построим тепловую карту со коэфицентами связностей

на которой более точно понятно во сколько раз наши данные связанны

In [ ]:
corrmat = train.corr()
plt.subplots(figsize=(15,9))
sns.heatmap(corrmat[(corrmat >= 0.4) | (corrmat < -0.4)], 
            cmap='viridis',
            vmax=1.0, vmin=-1.0,
            linewidth=0.1,
            annot=True,
            annot_kws={"size":8})

Обьеденим тестовые и обучающие данные для дальнейшего анализа

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.rate_difference.values
all_data = pd.concat((train, test), sort=True).reset_index(drop=True)
all_data.drop(['rate_difference'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

Посчитаем пропущеные данные

Таковых нет, что логично для этих данных

In [ ]:
all_data_na = (train.isnull().sum() / len(train)) * 100
# all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

Просмотрим смещение распределения признаков

коэфиценты хоршие)


In [ ]:
numeric_feats = train.dtypes[train.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = train[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

разделим данные обратно на тренировочные и тестовые

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

Хоть и набор данных был выбран не большой и не сложный, но именно это и заставило с ним помучаться-.-